In [ ]:
from numba import guvectorize, float64, int16
import numpy

In [ ]:
def restrict_2d(fine, coarse):
    
    J, I = fine.shape
    for j in range(2, J - 2 , 2):
        for i in range(2, I - 2, 2):
            coarse[j//2, i//2] = (1/16 * (
                        fine[j + 1, i + 1] + 
                        fine[j + 1, i - 1] +
                        fine[j - 1, i + 1] +
                        fine[j - 1, i - 1]) + 
                    1/8 * (
                    fine[j, i + 1] +
                    fine[j, i - 1] + 
                    fine[j - 1, i] + 
                    fine[j + 1, i]) +
                    1/4 * fine[j, i])

In [ ]:
@guvectorize([(int16[:,:], int16[:], int16[:,:])], '(n,n),(m)->(m,m)')
def restrict_2d_gvec(fine, size, coarse):
    
    J, I = fine.shape
    for j in range(2, J - 2 , 2):
        for i in range(2, I - 2, 2):
            coarse[j//2, i//2] = (1/16 * (
                        fine[j + 1, i + 1] + 
                        fine[j + 1, i - 1] +
                        fine[j - 1, i + 1] +
                        fine[j - 1, i - 1]) + 
                    1/8 * (
                    fine[j, i + 1] +
                    fine[j, i - 1] + 
                    fine[j - 1, i] + 
                    fine[j + 1, i]) +
                    1/4 * fine[j, i])
                    
    #return coarse
    

In [ ]:
from matplotlib import pyplot, cm
%matplotlib inline

In [ ]:
a = numpy.zeros((50, 50))
a[::2, ::2] = 1

In [ ]:
b = numpy.zeros_like(a[::2, ::2])

In [ ]:
from matplotlib import cbook

In [ ]:
filename = cbook.get_sample_data('jacksboro_fault_dem.npz', asfileobj=False)
with numpy.load(filename) as dem:
    z = dem['elevation']

In [ ]:
z = z[:,:344]

In [ ]:
z.dtype

In [ ]:
pyplot.figure(figsize=(10,10))
pyplot.imshow(z, interpolation='none', cmap=cm.viridis)

In [ ]:
%%timeit 
y = numpy.zeros_like(z[::2,::2])
restrict_2d(z, y)

In [ ]:
%%timeit
y = numpy.zeros_like(z[::2,::2])
restrict_2d_gvec(z, numpy.empty(y.shape[0], dtype=numpy.int16), y)

In [ ]:
pyplot.figure(figsize=(10,10))
y = numpy.zeros_like(z[::2,::2])
restrict_2d_gvec(z, numpy.empty(y.shape[0], dtype=numpy.int16), y)
pyplot.imshow(y, interpolation='none', cmap=cm.viridis)
z = y.copy()